Installere forskjellige pakker - fjern # for å ta inn linjen

In [146]:
# %pip install sympy


Bruker sympy

In [160]:
import sympy as sp
import pandas as pd
import numpy as np
from scipy.linalg import eigvals

## Oppgave 1A

Definerer en funksjon som tar inn funksjon, og retrunerer partiell derivert og gradienten

In [148]:
# Define symbolic variables
x, y = sp.symbols('x y')

Definerer funskjonene

In [149]:
def F(x, y):
    return 3*x**2 + 2*x*y + 4*y**2 - 6*x - 8*y + 5

def G(x, y):
    return (x**2 + 3*y**2)**2 - 2*x*y

In [150]:
def compute_partial_derivatives_gradients_hessa(func, vars):
    # Create the symbolic function
    f = func(x, y)

    # Compute partial derivatives
    derivatives = [sp.diff(f, var) for var in vars]

    # Compute the gradient
    gradient = [derivative.subs({x: 0, y: 0}) for derivative in derivatives]

    # Compute the Hessian matrix (second derivatives)
    hessian = sp.Matrix([[sp.diff(derivative1, var2) for var2 in vars] for derivative1 in derivatives])

    return derivatives, gradient, hessian

F (x,y) og G(x,y) er gitt av oppgaven

In [151]:
vars = (x, y)

derivatives_F, gradient_F, hessa_F = compute_partial_derivatives_gradients_hessa(F, vars)

print(derivatives_F)
print()
print (gradient_F)
print()
print(hessa_F)

[6*x + 2*y - 6, 2*x + 8*y - 8]

[-6, -8]

Matrix([[6, 2], [2, 8]])


In [152]:
derivatives_G, gradient_G , hessa_G= compute_partial_derivatives_gradients_hessa(G, vars)

print(derivatives_G)
print()
print (gradient_G)
print()
print(hessa_G)

[4*x*(x**2 + 3*y**2) - 2*y, -2*x + 12*y*(x**2 + 3*y**2)]

[0, 0]

Matrix([[12*x**2 + 12*y**2, 24*x*y - 2], [24*x*y - 2, 12*x**2 + 108*y**2]])


## Oppgave B

Har funksjonen en minimum, flere eller ingen? Forklar med et teoretisk eller geometrisk argument

For å bestemme om en funksjon har et minimum, må vi se på funksjonenes egenskaper

Ved å bruke den første deriverte, kan vi identifisere kritiske punkt

Det vil være Fx'(x,y) = 0 og Fy'(x,y) = 0, samme for G

Funksjonen gir en numpy array tilbake

In [153]:
def find_critical_points_fx_fy_zero(derivatives, vars):
    fx, fy = derivatives
    
    # Find critical points where fx = 0 and fy = 0
    critical_points = sp.solve([fx, fy], vars)
    
    return critical_points

In [154]:
critical_points_F = find_critical_points_fx_fy_zero(derivatives_F, (x, y))
critical_points_G = find_critical_points_fx_fy_zero(derivatives_G, (x, y))

print("Critical points for F:")
print(critical_points_F)
print()

print("Critical points for G:")
print(critical_points_G)

Critical points for F:
{x: 8/11, y: 9/11}

Critical points for G:
[(0, 0), (-3**(3/4)/6, -3**(1/4)/6), (3**(3/4)/6, 3**(1/4)/6), (-3**(3/4)*I/6, 3**(1/4)*I/6), (3**(3/4)*I/6, -3**(1/4)*I/6)]


Det er et kritisk punkt for F, for x og y
{x: 8/11, y: 9/11}
Kan sjekke

G har derimot mange, så det er ikke helt klart hva som er minimum

Omgjøring til numpy

In [155]:
gradient_F = np.array(gradient_F)
hessian_F = np.array(hessa_F)

Klassifisere Hessamatrisen

In [158]:
def klassifisereMinMaxSaddel(hessa_matrix, gradient):
    # Check if the gradient is zero
    det_hessian = np.linalg.det(hessa_matrix)

    # Check if the gradient is zero and the determinant is positive
    if np.all(gradient == 0) and det_hessian > 0:
        return "Local Minimum"
    elif np.all(gradient == 0) and det_hessian < 0:
        return "Local Maximum"
    else:
        return "Saddle Point (Indefinite)"

In [159]:
result = klassifisereMinMaxSaddel(hessian_F, gradient_F)
print(result)

UFuncTypeError: Cannot cast ufunc 'det' input from dtype('O') to dtype('float64') with casting rule 'same_kind'

F - burde ha et minimum siden 3x^2 og 4y^2 har positive fortegn

G - har positve verdier (x^2 + 3y^2)^2 , så burde ha bunnpunkter, for å følge den positive verdien 1 og 2 gradsfunksjoner

## Oppgave C

Finn minimum med klakulasjon (analytisk problemløsning)

## Oppgave 2a

## Oppgave 3a

In [185]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [186]:
svalbard = pd.read_excel("svalbard_met.xlsx")
svalbard = svalbard.dropna()

datacleaning

In [187]:
df = svalbard.rename(columns={'Tid(norsk normaltid)': 'dato', 'Homogenisert middeltemperatur (mnd)': 'temp'})[['dato', 'temp']]

# Convert the 'dato' column to datetime
df['dato'] = pd.to_datetime(df['dato'], errors='coerce')

# Filter the DataFrame to keep rows within the specified date range
df = df[(df['dato'] >= '2010-01-01') & (df['dato'] <= '2020-12-31')]

# df_filtered now contains only rows where 'dato' is between 2010-2020
df

,dato,temp
1336,2010-01-01,-7.2
1337,2010-02-01,-10.3
1338,2010-03-01,-15.8
1339,2010-04-01,-7.7
1340,2010-05-01,0.0
...,...,...
1463,2020-08-01,7.2
1464,2020-09-01,3.2
1465,2020-10-01,-2.2
1466,2020-11-01,-2.9


## Oppgave A

In [189]:
# Define the extended model function
def extended_model(x, a0, a1, a2, a3, a4):
    return a0 * np.sin(a1 * x + a2) + a3 + a4 * x

# Extract the data to be fitted
x_data = df['dato'].values
y_data = df['temp'].values

# Initial guess for the parameters
initial_guess = (2, 1, 0.5, 3, 0.1)

# Fit the model to the data
params, params_covariance = curve_fit(extended_model, x_data, y_data, p0=initial_guess)

# Extract the fitted parameters
a0_fit, a1_fit, a2_fit, a3_fit, a4_fit = params

# Generate fitted data using the model
y_fit = extended_model(x_data, a0_fit, a1_fit, a2_fit, a3_fit, a4_fit)

c:\Users\even\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


UFuncTypeError: ufunc 'multiply' cannot use operands with types dtype('float64') and dtype('<M8[ns]')

In [ ]:
# Plot the original data and the fitted curve
plt.figure(figsize=(8, 6))
plt.scatter(x_data, y_data, label='Data')
plt.plot(x_data, y_fit, 'r', label='Fitted Curve')
plt.xlabel('Year')
plt.ylabel('Temperature')
plt.legend()
plt.show()

# Print the fitted parameters
print(f"Fitted Parameters: a0={a0_fit}, a1={a1_fit}, a2={a2_fit}, a3={a3_fit}, a4={a4_fit}")